Instalação e importação do pySpark


In [ ]:
pip install pyspark

     |████████████████████████████████| 281.3 MB 33 kB/s 
     |████████████████████████████████| 198 kB 55.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=80422affe0d336992b8069b026c712aa911cd5cf2e89d52e11f249d116125180
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, TimestampType, DateType
from pyspark.sql.window import Window



---
Configurando SparkSession


In [ ]:
spark = (
    SparkSession.builder
      .master('local')
      .appName('marketing')
      .config('spark.ui.port', '4050')
      .getOrCreate()
)



---
Customização do Schema com StruckType


In [ ]:
customSchema = StructType([
    StructField("Index", IntegerType(), True),
    StructField("ID", StringType(), True),
    StructField("AnoNascimento", IntegerType(), True),
    StructField("Educacao", StringType(), True),
    StructField("EstadoCivil", StringType(), True),
    StructField("Renda", DoubleType(), True),
    StructField("CriancaNaCasa", IntegerType(), True),
    StructField("AdolescenteNaCasa", IntegerType(), True),
    StructField("DtInscricao", DateType(), True),
    StructField("UltimaCompra", IntegerType(), True),
    StructField("TotalVinho", IntegerType(), True),
    StructField("TotalFrutas", IntegerType(), True),
    StructField("TotalProdutosCarnes", IntegerType(), True),
    StructField("TotalProdutosPeixe", IntegerType(), True),
    StructField("TotalProdutosDoces", IntegerType(), True),        
    StructField("TotaltProdutosOuro", IntegerType(), True),
    StructField("QtComprasDesconto", IntegerType(), True),
    StructField("QtComprasWeb", IntegerType(), True),        
    StructField("QtComprasCatalogo", IntegerType(), True),
    StructField("QtComprasLoja", IntegerType(), True),
    StructField("QtVistasWebMes", IntegerType(), True),        
    StructField("AceitouCamapanha3", IntegerType(), True),
    StructField("AceitouCamapanha4", IntegerType(), True),
    StructField("AceitouCamapanha5", IntegerType(), True),        
    StructField("AceitouCamapanha1", IntegerType(), True),
    StructField("AceitouCamapanha2", IntegerType(), True),
    StructField("Reclamacao", StringType(), True),        
    StructField("Resposta", StringType(), True)
])

df = spark.read.load('/content/drive/MyDrive/Colab Notebooks/pre-projeto-analise de dados/marketing_dataframe.csv', format="csv", header="true", sep=',', schema=customSchema)



---
Deletar coluna desnecessária para análise


In [ ]:
df = df.drop("Index")



---
Renomeando colunas


In [ ]:
df = df.withColumnRenamed("Renda","Receita").withColumnRenamed("DtInscricao","DataInscricao")



---
Adicionando colunas extras que serão úteis para análise dos dados


Mostrar a quantidade de campanhas de marketing que foram aceitas por cada cliente

In [ ]:
df = df.withColumn("QtCamapanhaAceita", F.col("AceitouCamapanha3") + F.col("AceitouCamapanha4") + F.col("AceitouCamapanha5") + F.col("AceitouCamapanha1") + F.col("AceitouCamapanha2")) 
df.show()

+----+-------------+-----------+------------+-------+-------------+-----------------+-------------+------------+----------+-----------+-------------------+------------------+------------------+------------------+-----------------+------------+-----------------+-------------+--------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------+--------+-----------------+
|  ID|AnoNascimento|   Educacao| EstadoCivil|Receita|CriancaNaCasa|AdolescenteNaCasa|DataInscricao|UltimaCompra|TotalVinho|TotalFrutas|TotalProdutosCarnes|TotalProdutosPeixe|TotalProdutosDoces|TotaltProdutosOuro|QtComprasDesconto|QtComprasWeb|QtComprasCatalogo|QtComprasLoja|QtVistasWebMes|AceitouCamapanha3|AceitouCamapanha4|AceitouCamapanha5|AceitouCamapanha1|AceitouCamapanha2| Reclamacao|Resposta|QtCamapanhaAceita|
+----+-------------+-----------+------------+-------+-------------+-----------------+-------------+------------+----------+-----------+-------------------+-----

Mostra o total de compras feitas nos diferentes meios por cada cliente

In [ ]:
df = df.withColumn("TotalCompras", F.col("QtComprasDesconto") + F.col("QtComprasWeb") + F.col("QtComprasCatalogo") + F.col("QtComprasLoja")) 
df.show()

+----+-------------+-----------+------------+-------+-------------+-----------------+-------------+------------+----------+-----------+-------------------+------------------+------------------+------------------+-----------------+------------+-----------------+-------------+--------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------+--------+-----------------+------------+
|  ID|AnoNascimento|   Educacao| EstadoCivil|Receita|CriancaNaCasa|AdolescenteNaCasa|DataInscricao|UltimaCompra|TotalVinho|TotalFrutas|TotalProdutosCarnes|TotalProdutosPeixe|TotalProdutosDoces|TotaltProdutosOuro|QtComprasDesconto|QtComprasWeb|QtComprasCatalogo|QtComprasLoja|QtVistasWebMes|AceitouCamapanha3|AceitouCamapanha4|AceitouCamapanha5|AceitouCamapanha1|AceitouCamapanha2| Reclamacao|Resposta|QtCamapanhaAceita|TotalCompras|
+----+-------------+-----------+------------+-------+-------------+-----------------+-------------+------------+----------+-----------



---


Mostra a quantidade de menores de idade que residem junto aos clientes

In [ ]:
df = df.withColumn("MenorEmCasa", F.col("CriancaNaCasa") + F.col("AdolescenteNaCasa")) 
df.show()

+----+-------------+-----------+------------+-------+-------------+-----------------+-------------+------------+----------+-----------+-------------------+------------------+------------------+------------------+-----------------+------------+-----------------+-------------+--------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------+--------+-----------------+------------+-----------+
|  ID|AnoNascimento|   Educacao| EstadoCivil|Receita|CriancaNaCasa|AdolescenteNaCasa|DataInscricao|UltimaCompra|TotalVinho|TotalFrutas|TotalProdutosCarnes|TotalProdutosPeixe|TotalProdutosDoces|TotaltProdutosOuro|QtComprasDesconto|QtComprasWeb|QtComprasCatalogo|QtComprasLoja|QtVistasWebMes|AceitouCamapanha3|AceitouCamapanha4|AceitouCamapanha5|AceitouCamapanha1|AceitouCamapanha2| Reclamacao|Resposta|QtCamapanhaAceita|TotalCompras|MenorEmCasa|
+----+-------------+-----------+------------+-------+-------------+-----------------+-------------+-----------



---
Mostrar a Receita média separada pela escolaridade do consumidor


In [ ]:
df.groupby(F.col("Educacao")).agg(F.round(F.avg("Receita"), 2).alias("ReceitaMedia"), F.round(F.avg("TotalCompras"), 2).alias("MediaDeCompras")).show()

+-----------+------------+--------------+
|   Educacao|ReceitaMedia|MediaDeCompras|
+-----------+------------+--------------+
|  Doutorado|    56220.03|         15.84|
|  Graduacao|    52713.21|         14.99|
|     Mestre|    52859.73|          14.9|
|  Mestrando|    20306.26|          7.02|
|EnsinoMedio|    47633.19|          13.8|
+-----------+------------+--------------+





---


Mostrar Informação dos clientes que tem menores de idade morando na mesma casa

In [ ]:
df.groupBy(F.col("EstadoCivil"), F.col("Educacao")).agg(F.round(F.avg("Receita"), 2).alias("ReceitaMedia")).show()

+------------+-----------+------------+
| EstadoCivil|   Educacao|ReceitaMedia|
+------------+-----------+------------+
|    Solteiro|  Graduacao|    51322.18|
|       Viuvo|  Doutorado|    60288.08|
|UniaoEstavel|EnsinoMedio|    44736.41|
|    Solteiro|  Mestrando|    18238.67|
|    Solteiro|EnsinoMedio|    53673.94|
|UniaoEstavel|  Doutorado|    56041.42|
|       Viuvo|  Graduacao|    54976.66|
|    Solteiro|  Doutorado|    53314.61|
|       Viuvo|EnsinoMedio|     51392.2|
|UniaoEstavel|  Graduacao|    55758.48|
|       Viuvo|  Mestrando|     22123.0|
|UniaoEstavel|     Mestre|    52109.01|
|      Casado|EnsinoMedio|     46201.1|
|      Casado|  Mestrando|     21960.5|
|      Casado|  Doutorado|    58138.03|
|    Solteiro|     Mestre|    53530.56|
|  Divorciado|     Mestre|    50331.95|
|  Divorciado|  Mestrando|      9548.0|
|  Divorciado|  Doutorado|    53096.62|
|  Divorciado|EnsinoMedio|    49395.13|
+------------+-----------+------------+
only showing top 20 rows





---


Mostra os casos de pessoas casadas com menores morando em sua casa


In [ ]:
df.filter(F.col("MenorEmCasa") > 0).orderBy(F.col("EstadoCivil")).show()


+-----+-------------+-----------+-----------+-------+-------------+-----------------+-------------+------------+----------+-----------+-------------------+------------------+------------------+------------------+-----------------+------------+-----------------+-------------+--------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------+--------+-----------------+------------+-----------+
|   ID|AnoNascimento|   Educacao|EstadoCivil|Receita|CriancaNaCasa|AdolescenteNaCasa|DataInscricao|UltimaCompra|TotalVinho|TotalFrutas|TotalProdutosCarnes|TotalProdutosPeixe|TotalProdutosDoces|TotaltProdutosOuro|QtComprasDesconto|QtComprasWeb|QtComprasCatalogo|QtComprasLoja|QtVistasWebMes|AceitouCamapanha3|AceitouCamapanha4|AceitouCamapanha5|AceitouCamapanha1|AceitouCamapanha2| Reclamacao|Resposta|QtCamapanhaAceita|TotalCompras|MenorEmCasa|
+-----+-------------+-----------+-----------+-------+-------------+-----------------+-------------+-----------

---

Mostrar as pessoas casadas com receita anual de menos de 40000

In [ ]:
df.filter(F.col("EstadoCivil") == 'Casado').filter(F.col("Receita") < 40000.00).show()

+-----+-------------+-----------+-----------+-------+-------------+-----------------+-------------+------------+----------+-----------+-------------------+------------------+------------------+------------------+-----------------+------------+-----------------+-------------+--------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------+--------+-----------------+------------+-----------+
|   ID|AnoNascimento|   Educacao|EstadoCivil|Receita|CriancaNaCasa|AdolescenteNaCasa|DataInscricao|UltimaCompra|TotalVinho|TotalFrutas|TotalProdutosCarnes|TotalProdutosPeixe|TotalProdutosDoces|TotaltProdutosOuro|QtComprasDesconto|QtComprasWeb|QtComprasCatalogo|QtComprasLoja|QtVistasWebMes|AceitouCamapanha3|AceitouCamapanha4|AceitouCamapanha5|AceitouCamapanha1|AceitouCamapanha2| Reclamacao|Resposta|QtCamapanhaAceita|TotalCompras|MenorEmCasa|
+-----+-------------+-----------+-----------+-------+-------------+-----------------+-------------+-----------



---
Mostra  ranking percentual da relação entre Total de Compras e a Receita do usuário


In [ ]:
w0 = Window.partitionBy(F.col('TotalCompras')).orderBy('Receita')

In [ ]:
df.withColumn('percent_rank',F.percent_rank().over(w0)).show()

+-----+-------------+-----------+------------+--------+-------------+-----------------+-------------+------------+----------+-----------+-------------------+------------------+------------------+------------------+-----------------+------------+-----------------+-------------+--------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------+--------+-----------------+------------+-----------+--------------------+
|   ID|AnoNascimento|   Educacao| EstadoCivil| Receita|CriancaNaCasa|AdolescenteNaCasa|DataInscricao|UltimaCompra|TotalVinho|TotalFrutas|TotalProdutosCarnes|TotalProdutosPeixe|TotalProdutosDoces|TotaltProdutosOuro|QtComprasDesconto|QtComprasWeb|QtComprasCatalogo|QtComprasLoja|QtVistasWebMes|AceitouCamapanha3|AceitouCamapanha4|AceitouCamapanha5|AceitouCamapanha1|AceitouCamapanha2| Reclamacao|Resposta|QtCamapanhaAceita|TotalCompras|MenorEmCasa|        percent_rank|
+-----+-------------+-----------+------------+--------+---------



---

Mostrar a relação de nível educacional com o total de compras realizadas

In [ ]:
w1 = Window.partitionBy(F.col("Educacao")).orderBy("TotalCompras")

In [ ]:
df.withColumn("Ranking_Compras", F.row_number().over(w1)).select("Educacao", "TotalCompras", "Ranking_Compras").show()

+---------+------------+---------------+
| Educacao|TotalCompras|Ranking_Compras|
+---------+------------+---------------+
|Doutorado|           0|              1|
|Doutorado|           1|              2|
|Doutorado|           1|              3|
|Doutorado|           2|              4|
|Doutorado|           4|              5|
|Doutorado|           4|              6|
|Doutorado|           4|              7|
|Doutorado|           4|              8|
|Doutorado|           4|              9|
|Doutorado|           4|             10|
|Doutorado|           4|             11|
|Doutorado|           4|             12|
|Doutorado|           4|             13|
|Doutorado|           4|             14|
|Doutorado|           4|             15|
|Doutorado|           4|             16|
|Doutorado|           4|             17|
|Doutorado|           4|             18|
|Doutorado|           4|             19|
|Doutorado|           4|             20|
+---------+------------+---------------+
only showing top

In [ ]:
df.write.format("com.databricks.spark.csv").option("header", "true").option("inferschema", "true")\
    .option("delimiter", ",").save("/content/drive/MyDrive/Colab Notebooks/pre-projeto-analise de dados/marketing_campaign_pySpark")